# CROSS FADE TEST

In [1]:
%matplotlib inline

In [2]:
import numpy as np
from pydub import AudioSegment
from glob import glob
from matplotlib import pyplot as plt
import os
import IPython.display

In [3]:
import json
import os
import subprocess
import logging
from logging.config import dictConfig


__all__ = [
    'combine_audio_video',
    'validate_numeric',
    'create_log',
    'FireBaseConnector',
    'mov_to_mp4'
]


def mov_to_mp4(video_file,
               force: bool=False):
    """ Convert sample.MOV to sample.mp4 by ffmpeg

    `ffmpeg -i sample.MOV -vcodec h264 -acodec mp2 sample.mp4`

     Parameter
    --------------------
    video_file: str
        path to target video
    force: bool
        overwrite if it exists

     Return
    --------------------
    _file: str
        produced file
    _message: str
        message from ffmpeg
    """

    _file, _id = video_file.split('.')
    if not _id in ['mov', 'MOV']:
        raise ValueError('%s is not MOV' % video_file)
    _file += '.mp4'
    command = "ffmpeg -i %s -vcodec h264 -acodec mp2 %s" % (video_file, _file)
    if os.path.exists(_file):
        if force:
            os.system('rm -rf %s' % _file)
        else:
            return _file, ''
    try:
        output = subprocess.check_output(
            command, stderr=subprocess.STDOUT, shell=True, timeout=600,
            universal_newlines=True)
    except subprocess.CalledProcessError as exc:
        if os.path.exists(_file):
            os.system('rm -rf %s' % _file)

        raise ValueError("Status : FAIL", exc.returncode, exc.output)

    return _file, "Output: \n{}\n".format(output)


def combine_audio_video(video_file, audio_file, output_file):
    """ Combine audio data and video by ffmpeg

    `ffmpeg -i sample.mp4 -i sample.mp3 -vcodec copy sample_combined.mp4`

     Parameter
    --------------------
    video_file: str
        path to target video
    audio_file: str
        path to save audio wav file. shoud be end with ~.wav
    output_file
        file name for new combined video file

     Return
    --------------------
    _message: str
        output message from ffmpeg
    """

    if not (video_file.endswith('.mp4')):
        raise ValueError('unknown video format: %s' % video_file)
    if not (audio_file.endswith('.wav') or audio_file.endswith('.mp3')):
        raise ValueError('unknown audio format: %s' % audio_file)

    if not os.path.exists(video_file):
        raise ValueError('No video file at: %s' % video_file)

    if os.path.exists(output_file):
        os.system('rm -rf %s' % output_file)

    command = "ffmpeg -i %s -i %s -vcodec copy %s" % (video_file, audio_file, output_file)
    try:
        output = subprocess.check_output(
            command, stderr=subprocess.STDOUT, shell=True, timeout=600,
            universal_newlines=True)
    except subprocess.CalledProcessError as exc:
        if os.path.exists(output_file):
            os.system('rm -rf %s' % output_file)
        raise ValueError("Status : FAIL", exc.returncode, exc.output)

    return "Output: \n{}\n".format(output)


def validate_numeric(value: str,
                     default,
                     min_val,
                     max_val,
                     is_float=False):
    """ Validate numeric variable (for API parameter validation)

     Parameter
    ---------------
    value: str
        string value
    default: numeric
        default value if given value is None or ""
    min_val: numeric
        numeric value for minimum value
    max_val: numeric
        numeric value for maximum value
    is_float: bool
        use `float` if True else `int`

     Return
    ---------------
    flag: bool
        error flag True if there's any error
    value:
        value for the parameter
    """
    if value == '' or value is None:
        value = default
    try:
        if is_float:
            value = float(value)
        else:
            value = int(value)
    except ValueError:
        return False, f'Param must be a numeric value between "{min_val}" and "{max_val}"'

    if type(max_val) is not float and type(max_val) is not int:
        return False, 'max_val is not numeric: %s' % str(max_val)

    if type(min_val) is not float and type(min_val) is not int:
        return False, 'min_val is not numeric: %s' % str(min_val)

    if value > max_val or value < min_val:
        return False, f'Param must be between {min_val} and {max_val} but {value}'

    return True, value


def create_log():
    """ Logger

    Usage
    -------------------
    logger.info(message)
    logger.error(error)
    """
    logging_config = dict(
        version=1,
        formatters={
            'f': {'format': '%(asctime)s %(name)-12s %(levelname)-8s %(message)s'}
        },
        handlers={
            'h': {'class': 'logging.StreamHandler',
                  'formatter': 'f',
                  'level': logging.DEBUG}
        },
        root={
            'handlers': ['h'],
            'level': logging.DEBUG,
        },
    )
    dictConfig(logging_config)
    logger = logging.getLogger()
    return logger



In [4]:
""" Collection of functions to tune cutoff amplitude """

import numpy as np

VALID_METHOD_TYPES = ['percentile']
LOG = create_log()

__all__ = ['VALID_METHOD_TYPES', 'CutoffMethods']


class CutoffMethods:

    def __init__(self,
                 method_type='percentile'):
        if method_type not in VALID_METHOD_TYPES:
            raise ValueError('unknown `method_type`: %s not in %s' % (method_type, VALID_METHOD_TYPES))
        self.__method_type = method_type

    def get_cutoff_amp(self,
                       wave_data,
                       ratio: float=None):
        if self.__method_type in ['percentile']:
            if ratio is None:
                raise ValueError('method `percentile` requires `ratio`')
            return self.__percentile(wave_data, ratio)
        else:
            raise ValueError('unknown `method_type`: %s not in %s' % (self.__method_type, VALID_METHOD_TYPES))

    @staticmethod
    def __percentile(data, p):
        """ percentile method """
        p = np.clip(p, 0.0, 1.0)
        single_array_sorted = np.sort(np.abs(data))
        ind = int(np.floor(p * len(data)))
        val = single_array_sorted[min(ind, len(data) - 1)]
        LOG.debug('cutoff threshold (percentile method)')
        LOG.debug(' * percentile: %0.2f with %0.3f percentile' % (val, p))
        return int(val)


# New editor

In [33]:
""" Audio editor with cross fade """
import os
import numpy as np
from pydub import AudioSegment
from moviepy import editor

__all__ = ['AudioEditor']

# MOV (Apple's movie format), m4a (Apple's sound file format), mov file will be converted to mp4 file
VALID_FORMAT = ['mp3', 'wav', 'm4a', 'mp4', 'mov', 'MP3', 'WAV', 'M4A', 'MP4', 'MOV']
LOG = create_log()


class AudioEditor:
    """ Audio based video editor: (i) split video into audio and movie, (ii) process separately, (iii) combine
        `pydub.AudioSegment` for audio interface, and `moviepy.editor` for movie interface """

    def __init__(self,
                 file_path,
                 cutoff_method: str = 'percentile',
                 max_sample_length: int = None):
        """ Audio based video editor

         Parameter
        -------------
        file_path: str
            absolute path to file name
        cutoff_method: str
            cutoff method
        max_sample_length: int
            max sample length for audio file
        """
        audio, audio_stats, self.video, video_stats = self.load_audio(file_path)
        self.audio, self.wave_array_np_list = audio
        self.__audio_format, self.frame_rate, self.sample_width, self.channels = audio_stats
        self.__video_format, self.is_mov = video_stats
        self.__cutoff_instance = CutoffMethods(cutoff_method)
        self.length = len(self.wave_array_np_list[0])
        self.length_sec = self.length / self.frame_rate
        self.format = self.__audio_format if self.video is None else self.__video_format
        LOG.debug('audio info')
        LOG.debug(' * sample size     : %i' % self.length)
        LOG.debug(' * sample sec      : %0.3f' % self.length_sec)
        LOG.debug(' * channel         : %i' % self.channels)
        LOG.debug(' * frame rate      : %i' % self.frame_rate)
        LOG.debug(' * sample width    : %i' % self.sample_width)
        LOG.debug(' * audio_amplitude : %i (max), %i (min)' % (np.max(self.wave_array_np_list), np.min(self.wave_array_np_list)))
        if self.video is None:
            LOG.debug(' * no video')
        else:
            LOG.debug(' * video           : %s' % self.__video_format)
        if max_sample_length is not None and self.length > max_sample_length:
            raise ValueError('sample data exceeds max sample size: %i > %i' % (self.length, max_sample_length))

    def amplitude_clipping(self,
                           min_interval_sec: float,
                           ratio: float = 1.0,
                           crossfade_sec: float = 0.1):
        """ Amplitude-based truncation. In given audio signal, where every sampling point has amplitude
        less than `min_amplitude` and the length is greater than `min_interval`, will be removed. Note that
        even if the audio has multi-channel, first channel will be processed.

         Parameter
        ---------------
        min_interval_sec: int
            minimum interval of cutoff (sec)
        """
        assert min_interval_sec > 0 and crossfade_sec >= 0
        LOG.debug('start amplitude clipping')
        LOG.debug(' * min_interval_sec: %0.2f' % min_interval_sec)
        LOG.debug(' * ratio           : %0.2f' % ratio)
        LOG.debug(' * crossfade_sec   : %0.2f' % crossfade_sec)

        # pick up mono wave
        min_amplitude = self.__cutoff_instance.get_cutoff_amp(self.wave_array_np_list[0], ratio=ratio)
        min_interval = int(min_interval_sec * self.frame_rate)
        audio_signal_mask = np.array(np.abs(self.wave_array_np_list[0]) > min_amplitude)  # for audio edit
        LOG.debug('masked sample       : %i' % (self.length - np.sum(audio_signal_mask)))

        flg_deleting = False
        index = []
        pointer = 0
        deleted_sec = 0
        editing_audio = None
        editing_video = []
        last_edit_end = None
        last_cf_sec = 0
        prev_edit_point = None

        def _edit(_audio, _video, _last_edit_end, _last_cf_sec, _prev_edit_point, _current_edit_point=None):
            """ slicing is done by milliseconds: audio cross fade part will be overlapped but movie don't

            raw: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
            a: [1, 2, 3, 4, 5, 6], b: [7, 8, 9, 10, 11, 12]
            - audio cross fade within 2 sec: a_f = [1, 2, 3, 4, ff, ff, 9, 10, 11, 12], len(a_f) = 10
            - movie concat within 2 sec: m_f = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], len(m_f) = 12
            To align those incompatibility, movie will be concatenated by half the cross fade sec
               m_f = [1, 2, 3, 4, 5] + [8, 9, 10, 11, 12], len(m_f) = 10  * drop 6 from a, and 7 from b
            """
            if _current_edit_point is None:  # the final edit point
                if _prev_edit_point is not None:
                    start, end = _prev_edit_point
                    if _audio is None:  # only single keep part
                        _audio = self.audio[start * 1000:end * 1000]
                        if self.video is not None:
                            _video.append(self.video.subclip(start, end))
                    else:
                        _audio = _audio.append(self.audio[(start - _last_cf_sec) * 1000:end * 1000],
                                               crossfade=_last_cf_sec * 1000)
                        if self.video is not None:
                            _video.append(self.video.subclip(start - _last_cf_sec/2, end))
                return _audio, _video, 0, None, None

            start, end = _current_edit_point
            assert start <= end
            if end == start or (self.length_sec == end and start == 0):  # ignore the edit point
                return _audio, _video, _last_edit_end, _last_cf_sec, _prev_edit_point

            if _prev_edit_point is None:  # the first edit point
                return _audio, _video, 0, None, _current_edit_point

            p_start, p_end = _prev_edit_point
            _cf_sec = min((p_start - _last_edit_end)/2, crossfade_sec)

            if _audio is None:  # the second edit point
                _audio = self.audio[p_start * 1000:(p_end + _cf_sec) * 1000]
                if self.video is not None:
                    _video.append(self.video.subclip(p_start, p_end + _cf_sec/2))
            else:  # for other edit points
                _clip = self.audio[(p_start - _last_cf_sec) * 1000:(p_end + _cf_sec) * 1000]
                _audio = _audio.append(_clip, crossfade=_last_cf_sec * 1000)
                if self.video is not None:
                    _video.append(self.video.subclip(p_start - _last_cf_sec/2, p_end + _cf_sec/2))

            return _audio, _video, p_end, _cf_sec, _current_edit_point

        # loop for all the sample to get parts to keep
        for n, bool_mask in enumerate(audio_signal_mask):
            if bool_mask and flg_deleting:  # finish deleting chunk
                if min_interval <= len(index):  # delete the chunk if its longer than min length
                    delete_from = index[0] / self.frame_rate
                    delete_to = index[-1] / self.frame_rate
                    editing_audio, editing_video, last_edit_end, last_cf_sec, prev_edit_point = _edit(
                        editing_audio, editing_video, last_edit_end, last_cf_sec, prev_edit_point,
                        _current_edit_point=[pointer, delete_from])
                    pointer = delete_to
                    deleted_sec += delete_to - delete_from
                flg_deleting = False
                index = []
            if not bool_mask:
                flg_deleting = True
                index.append(n)

        if flg_deleting and min_interval <= len(index):
            end_point = index[0] / self.frame_rate
        else:
            end_point = self.length_sec

        editing_audio, editing_video, last_edit_end, last_cf_sec, prev_edit_point = _edit(
            editing_audio, editing_video, last_edit_end, last_cf_sec, prev_edit_point,
            _current_edit_point=[pointer, end_point])

        # final edit
        editing_audio, editing_video, _, _, _ = _edit(
            editing_audio, editing_video, last_edit_end, last_cf_sec, prev_edit_point,
            _current_edit_point=None)

        if editing_audio is None:
            LOG.debug('nothing to process')
            return False
        else:
            LOG.debug('processed: remove %0.3f sec (original audio was %0.2f sec)' % (deleted_sec, self.length_sec))
            # if fadeout_sec is not None:
            #     self.audio = editing_audio.fade_out(fadeout_sec)
            # else:
            self.audio = editing_audio

            # edit video
            if self.video is not None:
                LOG.debug('process video: * %i sub videos' % len(editing_video))
                self.video = editor.concatenate_videoclips(editing_video)
            return True

    # def vis_amplitude_clipping(self,
    #                            ratio: float = 1.0,
    #                            path_to_save: str = None):
    #     wave = self.wave_array_np_list[0]
    #     self.__cutoff_instance.visualize_cutoff_threshold(
    #         wave_data=wave,
    #         frame_rate=self.frame_rate,
    #         ratio=ratio,
    #         path_to_save=path_to_save
    #     )

    def write(self, _file_prefix: str):
        """ Write audio to file (format should be same as the input audio file)

         Parameter
        ----------
        _file_prefix: file prefix to save
        """

        def validate_path(__path):
            if os.path.exists(__path):
                os.remove(__path)
            if not os.path.exists(os.path.dirname(__path)):
                os.makedirs(os.path.dirname(__path), exist_ok=True)
            return __path

        _file_audio = validate_path(_file_prefix + '.%s' % self.__audio_format)
        LOG.debug(' * save audio to %s' % _file_audio)
        self.audio.export(_file_audio, format=self.__audio_format)

        if self.video is not None:

            # _file_audio = validate_path(_file.replace('.%s' % self.__video_format, '.%s' % self.__audio_format))
            LOG.debug(' * save audio to %s' % _file_audio)
            self.audio.export(_file_audio, format=self.__audio_format)

            _file_video_no_audio = validate_path(_file_prefix + '_no_audio.%s' % self.__video_format)
            _file_video = validate_path(_file_prefix + '.%s' % self.__video_format)

            LOG.debug(' * save video to %s' % _file_video_no_audio)
            self.video.write_videofile(_file_video_no_audio)
            LOG.debug(' * embed audio to video, save to %s' % _file_video)
            combine_audio_video(video_file=_file_video_no_audio, audio_file=_file_audio, output_file=_file_video)
            return _file_video
        else:
            return _file_audio

    @staticmethod
    def load_audio(file_path):
        """ Load audio file

         Parameter
        -----------
        file_path: str
            path to audio file, should be in VALID_FORMAT

         Return
        -----------
        audio: tuple
            (pydub.AudioSegment instance of audio, list of numpy array audio signal for each channel)
        audio_stats: tuple
            audio_format, frame_rate, sample_width, channels, channel size
        video:
            moviepy.editor instance of video
        video_stats: tuple
            convert_mov (bool if mov file has been converted)
        """
        # check file
        if not os.path.exists(file_path):
            raise ValueError('No file: %s' % file_path)

        # validate sound file (load as AudioSegment object
        video, video_format, convert_mov = None, None, False
        if file_path.endswith('.wav') or file_path.endswith('.WAV'):
            audio_format = 'wav'
            audio = AudioSegment.from_wav(file_path)
        elif file_path.endswith('.mp3') or file_path.endswith('.MP3'):
            audio_format = 'mp3'
            audio = AudioSegment.from_mp3(file_path)
        elif file_path.endswith('.m4a') or file_path.endswith('.M4A'):
            audio_format = 'm4a'
            audio = AudioSegment.from_file(file_path, audio_format)
        elif file_path.endswith('.mp4') or file_path.endswith('.MP4') \
                or file_path.endswith('.mov') or file_path.endswith('.MOV'):
            if file_path.endswith('.mov') or file_path.endswith('.MOV'):
                # mov format needs to be converted to mp4 firstly
                LOG.debug('convert MOV to mp4')
                file_path, _ = mov_to_mp4(file_path)
            audio_format = 'mp3'
            video_format = 'mp4'
            audio = AudioSegment.from_file(file_path)
            video = editor.VideoFileClip(file_path)
            convert_mov = True
        else:
            raise ValueError('unknown format %s (valid format: %s)' % (file_path, VALID_FORMAT))

        # numpy array from array.array object
        wave_array_np = np.array(audio.get_array_of_samples())
        # if stereo (channel > 1)
        if audio.channels != 1:
            if audio.channels > 2:
                raise ValueError('audio has more than two channel: %i' % audio.channels)
            wave_array_np_left = wave_array_np[0:len(wave_array_np):2]
            wave_array_np_right = wave_array_np[1:len(wave_array_np):2]
            wave_array_np_list = [wave_array_np_left, wave_array_np_right]
        else:
            wave_array_np_list = [wave_array_np]

        # information of audio file
        audio_stats = (audio_format, audio.frame_rate, audio.sample_width, audio.channels)
        video_stats = (video_format, convert_mov)
        return (audio, wave_array_np_list), audio_stats, video, video_stats

# Compare - mp4 -

In [37]:
sample_file = '/Users/ushioasahi/nitro_editor_data/sample_files/sample_6.MOV'

_editor = AudioEditor(sample_file)
_editor.amplitude_clipping(0.12, 0.9, 0.0)
_editor.write('../sample_files/sample_6_edit')

_editor = AudioEditor(sample_file)
_editor.amplitude_clipping(0.12, 0.9, 0.1)
_editor.write('../sample_files/sample_6_edit_xfade')

2020-02-09 16:48:36,649 root         DEBUG    convert MOV to mp4
2020-02-09 16:48:43,255 root         DEBUG    audio info
2020-02-09 16:48:43,257 root         DEBUG     * sample size     : 543263
2020-02-09 16:48:43,258 root         DEBUG     * sample sec      : 12.319
2020-02-09 16:48:43,259 root         DEBUG     * channel         : 1
2020-02-09 16:48:43,260 root         DEBUG     * frame rate      : 44100
2020-02-09 16:48:43,261 root         DEBUG     * sample width    : 2
2020-02-09 16:48:43,265 root         DEBUG     * audio_amplitude : 9062 (max), -8460 (min)
2020-02-09 16:48:43,266 root         DEBUG     * video           : mp4
2020-02-09 16:48:43,269 root         DEBUG    start amplitude clipping
2020-02-09 16:48:43,269 root         DEBUG     * min_interval_sec: 0.12
2020-02-09 16:48:43,270 root         DEBUG     * ratio           : 0.90
2020-02-09 16:48:43,271 root         DEBUG     * crossfade_sec   : 0.00
2020-02-09 16:48:43,300 root         DEBUG    cutoff threshold (percen

Moviepy - Building video ../sample_files/sample_6_edit_no_audio.mp4.
MoviePy - Writing audio in sample_6_edit_no_audioTEMP_MPY_wvf_snd.mp3


t:   7%|▋         | 14/199 [00:00<00:01, 135.82it/s, now=None]     

MoviePy - Done.
Moviepy - Writing video ../sample_files/sample_6_edit_no_audio.mp4



2020-02-09 16:48:50,065 root         DEBUG     * embed audio to video, save to ../sample_files/sample_6_edit.mp4


...

t:   8%|▊         | 17/221 [00:00<00:01, 164.38it/s, now=None]     

MoviePy - Done.
Moviepy - Writing video ../sample_files/sample_6_edit_xfade_no_audio.mp4



2020-02-09 16:48:58,832 root         DEBUG     * embed audio to video, save to ../sample_files/sample_6_edit_xfade.mp4


Moviepy - Done !
Moviepy - video ready ../sample_files/sample_6_edit_xfade_no_audio.mp4


'../sample_files/sample_6_edit_xfade.mp4'

In [ ]:
sample_file = '../sample_files/sample_4.mp4'

_editor = AudioEditor(sample_file)
_editor.amplitude_clipping(0.12, 0.9, 0.0)
_editor.write('../sample_files/sample_4_edit')

_editor = AudioEditor(sample_file)
_editor.amplitude_clipping(0.12, 0.9, 0.1)
_editor.write('../sample_files/sample_4_edit_xfade')

In [35]:
sample_file = '/Users/ushioasahi/nitro_editor_data/sample_files/sample_7.MOV'

_editor = AudioEditor(sample_file)
_editor.amplitude_clipping(0.12, 0.9, 0.0)
_editor.write('../sample_files/sample_7_edit')

_editor = AudioEditor(sample_file)
_editor.amplitude_clipping(0.12, 0.9, 0.1)
_editor.write('../sample_files/sample_7_edit_xfade')

2020-02-09 16:37:04,752 root         DEBUG    convert MOV to mp4
2020-02-09 16:37:05,365 root         DEBUG    audio info
2020-02-09 16:37:05,367 root         DEBUG     * sample size     : 3520031
2020-02-09 16:37:05,368 root         DEBUG     * sample sec      : 79.819
2020-02-09 16:37:05,369 root         DEBUG     * channel         : 1
2020-02-09 16:37:05,371 root         DEBUG     * frame rate      : 44100
2020-02-09 16:37:05,372 root         DEBUG     * sample width    : 2
2020-02-09 16:37:05,382 root         DEBUG     * audio_amplitude : 7643 (max), -6833 (min)
2020-02-09 16:37:05,383 root         DEBUG     * video           : mp4
2020-02-09 16:37:05,390 root         DEBUG    start amplitude clipping
2020-02-09 16:37:05,393 root         DEBUG     * min_interval_sec: 0.12
2020-02-09 16:37:05,394 root         DEBUG     * ratio           : 0.90
2020-02-09 16:37:05,395 root         DEBUG     * crossfade_sec   : 0.10
2020-02-09 16:37:05,535 root         DEBUG    cutoff threshold (perce

Moviepy - Building video ../sample_files/sample_7_edit_xfade_no_audio.mp4.
MoviePy - Writing audio in sample_7_edit_xfade_no_audioTEMP_MPY_wvf_snd.mp3


t:   0%|          | 7/1505 [00:00<00:26, 56.06it/s, now=None]        

MoviePy - Done.
Moviepy - Writing video ../sample_files/sample_7_edit_xfade_no_audio.mp4



2020-02-09 16:39:51,308 root         DEBUG     * embed audio to video, save to ../sample_files/sample_7_edit_xfade.mp4


Moviepy - Done !
Moviepy - video ready ../sample_files/sample_7_edit_xfade_no_audio.mp4


'../sample_files/sample_7_edit_xfade.mp4'

# Compare - mp3 -

In [20]:
sample_file = '../sample_files/sample_1.wav'

_editor = AudioEditor(sample_file)
_editor.amplitude_clipping(0.1, 0.7, 0.0)
_editor.write('../sample_files/sample_1_edit')

_editor = AudioEditor(sample_file)
_editor.amplitude_clipping(0.1, 0.7, 0.1)
_editor.write('../sample_files/sample_1_edit_xfade')

2020-02-09 11:48:40,950 root         DEBUG    audio info
2020-02-09 11:48:40,951 root         DEBUG     * sample size     : 145001
2020-02-09 11:48:40,951 root         DEBUG     * sample sec      : 18.125
2020-02-09 11:48:40,953 root         DEBUG     * channel         : 1
2020-02-09 11:48:40,954 root         DEBUG     * frame rate      : 8000
2020-02-09 11:48:40,956 root         DEBUG     * sample width    : 2
2020-02-09 11:48:40,958 root         DEBUG     * audio_amplitude : 26460 (max), -22217 (min)
2020-02-09 11:48:40,959 root         DEBUG     * no video
2020-02-09 11:48:40,960 root         DEBUG    start amplitude clipping
2020-02-09 11:48:40,962 root         DEBUG     * min_interval_sec: 0.10
2020-02-09 11:48:40,963 root         DEBUG     * ratio           : 0.70
2020-02-09 11:48:40,964 root         DEBUG     * crossfade_sec   : 0.00
2020-02-09 11:48:40,975 root         DEBUG    cutoff threshold (percentile method)
2020-02-09 11:48:40,977 root         DEBUG     * percentile: 252

'../sample_files/sample_1_edit_xfade_fadeout.wav'

In [21]:
IPython.display.Audio(sample_file)

In [24]:
IPython.display.Audio('../sample_files/sample_1_edit.wav')

In [25]:
IPython.display.Audio('../sample_files/sample_1_edit_xfade.wav')